## Auto ML

In [ ]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
train_df = pd.read_csv('data/240822_Rockhead_Seoul_train.csv')
test_df = pd.read_csv('data/240822_Rockhead_Seoul_test.csv')

In [ ]:
train = TabularDataset(train_df.drop(['depth_start'], axis=1))
test = TabularDataset(test_df.drop(['depth_start'], axis=1))

In [ ]:
predictor = TabularPredictor(label='bedrock_head', eval_metric='roc_auc',).fit(train)

In [ ]:
ld_board = predictor.leaderboard(train, silent=True)

ld_board

In [ ]:
pred_y = predictor.predict(test)